# This is the notebook for training SVC for all models [each drumtype]

In [ ]:
 script_idea = """
SCRIPT for training SVC for each drumtype (6 models in total), also contains a few function that might be useful for other models.
You still need to perform a refactor on this.

We are not using test ratio in this opportunity; we'll leave test results
for the heldout dataset. 
"""
print(script_idea)



SCRIPT for training SVC for each drumtype (6 models in total), also contains a few function that might be useful for other models.
You still need to perform a refactor on this.

We are not using test ratio in this opportunity; we'll leave test results
for the heldout dataset. 



In [ ]:
MYSEED = 1995
# preffix for the model name
models_name = "v2_svc_model"
# available drumtypes for training
available_labels_list = ['KD','SD','HH','TT','CY','OT']
# instrument that we will train a model for
target_label = "HH"

#params
validation_ratio = 0.2 # you should use then after
train_ratio = 1-validation_ratio

In [ ]:
import librosa
import warnings
from librosa import display
import matplotlib.pyplot as plt
from google.colab import drive
import os,sys,re,pandas as pd,numpy as np
import seaborn as sns
from scipy.io import wavfile
import math
import glob
from sympy import Interval
import logging 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from joblib import dump, load
from sklearn.model_selection import GridSearchCV

ROOT_DIR = "/content/drive"
drive.mount(ROOT_DIR, force_remount = True)
# Don't forget to type My Drive before the whole path
MUSIC_DIR = os.path.join(ROOT_DIR,'My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums')
MUSIC_DIR

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


Mounted at /content/drive


'/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums'

In [ ]:
# download trainmodel_utils.py and import it 
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/tesis_esp')
import trainmodel_utils as tmu

# 1) Define some functions

In [ ]:
def logprint(string):
  "Print and log with the same function"
  print(string)
  logging.info(string)


In [ ]:
def save_model(model, model_path):
    "Dump the model as joblib"
    dump(model, model_path) 
def load_model(model_path):
  "Load the model using joblib"
  clf = load(model_path) 
  return clf

def flatten_numpy_data(numpy_array:np.array):
  H,W,C = numpy_array.shape
  newshape = W*C
  numpy_array = numpy_array.reshape(H,newshape)
  return numpy_array


def map_stringlabels_to_binary(y):
  """
  Function for mapping binary string labels into 1s and 0s:
    If OTHER, the label will be 0 ; otherwise it'll be 1.
    Must have 2 labels -> Target label (IE KD,SD,HH) and OTHER.
  """
  y = np.array(y)
  n_uniquevalues = len(np.unique(y))
  if n_uniquevalues!=2: 
    raise ValueError("[ERROR] y must have exactly 2 unique values: OTHER and the target_label")
  # parameters
  condlist = [y=="OTHER"]
  replacelist = [0]
  default_replacement = 1 
  y_mapped = np.select(condlist,replacelist,default = default_replacement)
  return y_mapped

def run_grid_search_and_save_info(x_train, y_train, x_val, y_val, modelpath2save, n_jobs):
  """
  perform GS 5 fold CV ; testing on validation and testing aswelll  ( you dont need to test on val so val set could be  0 and add that to train test ; for not let it be like this)
  x_train,... train data val data test data ;  y_train refers to train labels
  modelpath2save -> must be .joblib
  return -> Nothing
  """
  global clf_gs 
  _, suffix_path = os.path.split(modelpath2save)
  if not ".joblib" in suffix_path:
    raise ValueError("modelpath2save must be .joblib")

  # define space params and model
  mycostlist = [0.01,0.1,1,2,5]
  parameters = [
                {'kernel':['linear'], 'C':mycostlist}
                ,{'kernel':['poly'],'degree':[1,2,3,5],'C':mycostlist}
                ,{'kernel':['rbf'],'C':mycostlist,'gamma':['scale','auto',0.001,0.01,0.02,0.03,0.1,1]}
                ]

  svc = SVC()

  # You can use any of this functions in order to get "best result"; since 
  # were using binary classif ; f1 might be a good metric
  # https://scikit-learn.org/stable/modules/model_evaluation.html Y
  # train, predict and eval
  clf_gs = GridSearchCV(svc, parameters,verbose=100,cv=5,n_jobs=n_jobs, scoring='f1')
  clf_gs.fit(x_train, y_train)
  y_train_predict = clf_gs.predict(x_train)
  y_val_predict = clf_gs.predict(x_val)
  classif_report_train = classification_report(y_pred = y_train_predict, y_true = y_train)
  classif_report_val = classification_report(y_pred = y_val_predict, y_true = y_val)

  # logs
  logprint("===================CLASIF REPORT WITH HP, TRAIN===============")
  logprint(classif_report_train)
  logprint("===================CLASIF REPORT WITH HP, VAL===============")
  logprint(classif_report_val)
  logprint("============ BEST ESTIMATOR ===")
  logprint(str(clf_gs.best_estimator_))
  # save
  save_model(clf_gs,modelpath2save)

# 2) Define paths, train the models, save them and show results

In [ ]:
# ncores
ncores = os.cpu_count()
# define paths
AUDIO_DIR = os.path.join(MUSIC_DIR,'audio','drum_only')
ANNOTATIONS_DIR = os.path.join(MUSIC_DIR,'annotations','class')
EXP_PIPE_DATA = os.path.join(MUSIC_DIR,"pipe005_multiplemodelsdata_corrected_over60")
EXP_PIPE_DATA_TRAIN = os.path.join(EXP_PIPE_DATA,"train")
EXP_PIPE_DATA_TEST = os.path.join(EXP_PIPE_DATA,"test")

In [ ]:
# For training each of the models us
# for target_label in target_labels_desired:
file_paths_txt = glob.glob(EXP_PIPE_DATA_TRAIN+"/*.txt")
file_paths_npy = glob.glob(EXP_PIPE_DATA_TRAIN+"/*.npy")
file_paths_txt_test = glob.glob(EXP_PIPE_DATA_TEST+"/*.txt")
file_paths_npy_test = glob.glob(EXP_PIPE_DATA_TEST+"/*.npy")
# SACVE RESULTS TO SOMEWHERE!
## define paths
modelpath2save = os.path.join(EXP_PIPE_DATA,f"{models_name}_{target_label}.joblib")
logfile_path = os.path.join(EXP_PIPE_DATA,f'{models_name}_{target_label}_logfile.log')
logging.basicConfig(filename = logfile_path ,
                    format='%(asctime)s %(levelname)s:%(message)s',
                    level=logging.INFO, datefmt='%m/%d/%Y %I:%M:%S %p')

# load labels and songsspectrograms
file_paths_txt,file_paths_npy = tmu.filter_path_for_target_label(file_paths_txt,file_paths_npy,target_label)
labels_list,data_npy_list = tmu.load_labels_and_data_from_npy(file_paths_npy)
data = tmu.merge_numpy_data(data_npy_list)
labels = tmu.merge_labels_data(labels_list)
labels = map_stringlabels_to_binary(labels)
assert data.shape[0] == len(labels)
logprint(f" Dataset is of {data.shape[0]} onsets, \n with a 33x9 (64-> window size, strides of 16 to a padded signal of len 128")
# Flatten and split
data_flatten = flatten_numpy_data(data)
x_train, x_val,y_train,y_val = train_test_split(data_flatten,labels,
                                                test_size = validation_ratio,
                                                random_state = MYSEED,
                                                stratify = None)
# run gs cv
run_grid_search_and_save_info(x_train, y_train, x_val, y_val,
                              modelpath2save,
                              n_jobs = ncores)
print(f"Unique labels {list(set(labels))}")

User must verify if the following files correspoond eaco other :
 /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/pipe005_multiplemodelsdata_corrected_over60/train/MusicDelta_Disco_class__HH.txt 
 /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/pipe005_multiplemodelsdata_corrected_over60/train/MusicDelta_Disco_class__HH.npy
User must verify if the following files correspoond eaco other :
 /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/pipe005_multiplemodelsdata_corrected_over60/train/MusicDelta_Britpop_class__HH.txt 
 /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/pipe005_multiplemodelsdata_corrected_over60/train/MusicDelta_Britpop_class__HH.npy
User must verify if the following files correspoond eaco other :
 /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/pipe005_multiplemodelsdata_corrected_over60/train/Music

# 3) Results from the HPO (hyperparameteroptimization)

In [ ]:
gs_results = pd.DataFrame(clf_gs.cv_results_)
gs_results.to_csv( os.path.join(EXP_PIPE_DATA,f"{models_name}_{target_label}_experiments.csv") )
gs_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_degree,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,16.054431,0.703780,3.194167,0.077003,0.01,linear,NaN,NaN,"{'C': 0.01, 'kernel': 'linear'}",0.940937,0.951807,0.938017,0.945455,0.932238,0.941691,0.006632,1
1,11.378320,0.322385,2.076079,0.341410,0.1,linear,NaN,NaN,"{'C': 0.1, 'kernel': 'linear'}",0.932238,0.949900,0.940206,0.941650,0.935484,0.939896,0.006023,2
2,11.096542,0.786957,1.690076,0.121108,1,linear,NaN,NaN,"{'C': 1, 'kernel': 'linear'}",0.920833,0.943320,0.927835,0.939516,0.931727,0.932646,0.008058,5
3,10.618683,0.598629,1.647775,0.160193,2,linear,NaN,NaN,"{'C': 2, 'kernel': 'linear'}",0.922105,0.943320,0.919588,0.933063,0.931452,0.929905,0.008481,8
4,10.657849,0.562145,1.651514,0.138802,5,linear,NaN,NaN,"{'C': 5, 'kernel': 'linear'}",0.922105,0.943320,0.921488,0.933063,0.931452,0.930285,0.008041,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,58.919683,1.145165,15.400448,0.401164,5,rbf,NaN,0.01,"{'C': 5, 'gamma': 0.01, 'kernel': 'rbf'}",0.856481,0.845266,0.868778,0.894855,0.847926,0.862661,0.018065,14
61,61.573924,0.432189,16.817886,0.500693,5,rbf,NaN,0.02,"{'C': 5, 'gamma': 0.02, 'kernel': 'rbf'}",0.799031,0.801932,0.810552,0.852535,0.789346,0.810679,0.021996,18
62,62.545469,0.331634,17.801994,0.413377,5,rbf,NaN,0.03,"{'C': 5, 'gamma': 0.03, 'kernel': 'rbf'}",0.757576,0.747475,0.786241,0.826603,0.744304,0.772440,0.030854,23
63,66.478035,0.995137,20.207102,0.600315,5,rbf,NaN,0.1,"{'C': 5, 'gamma': 0.1, 'kernel': 'rbf'}",0.633333,0.648501,0.652055,0.715405,0.673854,0.664630,0.028499,46
